In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import db
import sqlalchemy
import random
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
engine = sqlalchemy.create_engine("sqlite:///books.db")
db.Base.metadata.create_all(engine)
Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

In [4]:
similarity_function = lambda x, y: 1 - abs(x - y)
similarity_function = cosine_similarity
similarity_function = lambda x, y: np.dot(x, np.transpose(y))

# Recommendations

In [7]:
def compute_score(scoring_book, rated_book, rating, cosine_weight = 0.5, cluster_weight = 0.5):
    sb = session.query(db.Book).filter(db.Book.id == scoring_book).first()
    rb = session.query(db.Book).filter(db.Book.id == scoring_book).first()
    score = rating * (cosine_weight * similarity_function(sb, rb))
    return score

In [8]:
compute_score(24, 20, 1)

NoForeignKeysError: Could not determine join condition between parent/child tables on relationship Book.summary - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

In [ ]:
clusters = db.Cluster.all()

In [ ]:
def compute_score(embedding1, embedding2):
    return cosine_similarity(embedding1.array.reshape(1, -1), embedding2.array.reshape(1, -1))[0][0]

In [ ]:
def get_candidates(books):
    scores = []
    for cluster in clusters:
        score = 0
        for book in books:
            score += book[0] * compute_score(book[1], cluster.centroid)
        scores.append((cluster, score))
    return sorted(scores, key=lambda x: x[1], reverse=True)[0:10]

In [ ]:
def get_recommendations(books):
    candidates = get_candidates(books)
    recommendations = []
    for candidate in candidates:
        score = 0
        for book in books:
            score += book[0] * compute_score(book[1], candidate)
        recommendations.append((candidate, score))
    return sorted(recommendations, key=lambda x: x[1], reverse=True)[0:5]

In [ ]:
get_recommendations([
    (1, 1),
    (-1, 2),
    (1, 3)
])